# Python OOP - A simple game of blackjack

<img src="https://www.nydailynews.com/resizer/zwVN_CR8A-dPHjwaoVWpLrsqLEY=/1200x0/top/cloudfront-us-east-1.images.arcpublishing.com/tronc/3DHGXKFDEBBYPIXKJJHREM5IQE.jpg" width=500>


_- Unrelated Jack Black image_

In [1]:
import random
# We'll need it later

In [2]:
# Let's start with the begining.
# Cards are the most elemental object we will use.

class Card:
    # Let's construct our cards with a value and a suit.
    def __init__(self,value,suit):
        self.value=value
        self.suit=suit
        # We also define an `actual_value` attribute to make it easier for us in the future to sum
        # and calculate points.
        vals = {"J":10,"Q":10,"K":10,"A":1}
        self.actual_value = value if isinstance(value,int) else vals.get(value,value)
    
    def __repr__(self):
        """
        Making our Card objects a little easier on the eyes, having they just show the value and suit.
        The `rjust` method of strings makes it so that they all span 3 characters length.
        """
        if self.suit in ["♥","♦"]:
            return f"\x1b[31m{str(self.value).rjust(2,' ')}{self.suit}\x1b[0m"
        return f"{str(self.value).rjust(2,' ')}{self.suit}"
    
    def __str__(self):
        # For printig, we will also put the card representation in a little box
        # Just to make it look a little cool.
        return f"┌───┐\n|{self.__repr__()}|\n└───┘"
    
    # The addition methods are overwritten here to allow us to do things such as `sum(hand_of_cards)`
    # on the future and just abstract a little more on having to calculate the values.
    def __add__(self,other):
        return self.actual_value + other
    
    def __radd__(self,other):
        return self.actual_value + other
    
    # So we can compare the values of two cards
    def __eq__(self,other):
        if isinstance(other,Card):
            if self.value != other.value:
                return False
            if self.suit != other.suit:
                return False
            return True
        elif isinstance(other,int):
            return self.actual_value == other
    def __lt__(self,other):
        return self.actual_value < other
    def __gt__(self,other):
        return self.actual_value > other 

In [3]:
# Why not group cards on a deck object?

class Deck:
    def __init__(self):
        # We initiate the `cards` attribute and fill it with all the 52 card of the deck
        self.cards = [Card(value,suit)\
                      for suit in ["♠","♥","♦","♣"]\
                      # for each of the four suits
                      for value in ["A"] + list(range(2,11)) + ["J","Q","K"] ]
                      # For each value in ['A',2,3,4,5,6,7,8,9,10,'J','Q','K']
                      # We create a list of Card objects
                
    def shuffle(self):
        # We use this nifty little function to shuffle our deck
        random.shuffle(self.cards)
        
    def draw(self):
        # We check whether there are further cards to avoid an IndexError from popping an empty list
        if self.cards:
            
            return self.cards.pop(0)
        else:
            return "No more cards!"
    
    def __len__(self):
        # So we can calculate the len of a deck object
        return len(self.cards)
    
    def remove(self,card):
        if card in self.cards:
            self.cards.remove(card)
    
    def __repr__(self):
        # We represent a deck as the representation of the list of cards.
        return self.cards.__repr__()
    
    def __iter__(self):
        # So we can iterate through our cards
        return iter(self.cards)

In [4]:
# And some players!
class Player:
    def __init__(self,name="Player"):
        self.name=name
        self.hand=[]
        
    def take_card(self,card):
        self.hand.append(card)
        
    def hit(self):
        hit = input(f"{self.name}, do you want another card? [y|n]")
        if hit.lower() == "y":
            return True
        elif hit.lower() == "n":
            return False
        else:
            print("Wrong input. Try again...")
            return self.hit()
    
    def __repr__(self):
        return self.name
    
    def __str__(self, hand=None):
        # Printing a player will print all the cards on his hand
        if not hand:
            hand=self.hand
        return  f"{self.name}\n"+\
                "\n".join(\
                ["".join(line) for line in zip(\
                *[str(card).split("\n") for card in hand])])

In [5]:
# A CPU is a player that plays itself.
class CPU(Player):
    def __init__(self,name="CPU"):
        super().__init__(name)
        self.card_counting = Deck()
    
    def count_cards(self,game):
        # The computer begins with a card_counting deck
        # He removes the card on his own hand
        for card in self.hand:
            self.card_counting.remove(card)
        # He then removes the cards he can see from the other players.
        for player in game.players:
            # First card is hidden. Computer will not cheat
            for card in player.hand[1:]:
                self.card_counting.remove(card)
                
    def prob(self):
        # Computer calculates the probability of a new card busting, giving him
        # exactly 21 or under 21.
        # Everytime the probability of a good result is larger than a bust
        # it will take another card
        target = 21 - sum(self.hand)
        self.probabilities = {
            "bust": len([card for card in self.card_counting if card > target])\
                    /(len(self.card_counting)-1),
            21: len([card for card in self.card_counting if card == target])\
                    /(len(self.card_counting)-1),
            "under": len([card for card in self.card_counting if card < target])\
                    /(len(self.card_counting)-1)
        }
        
    def hit(self):
        print(f"{self.name}, do you want another card? [y|n]")
        # Computer calculates the probabilities
        self.prob()
        # And acts acordingly
        pros = self.probabilities[21] + self.probabilities["under"]
        cons = self.probabilities["bust"]
        hit = pros > cons
        if hit:
            print("Yes")
        else:
            print("No")
        return hit
    
    def __str__(self):
        # First card of computer is hidden, otherwise player would be cheating
        return super().__str__([Card("▓▓","▓"),*self.hand[1:]])
    
    def reveal(self):
        return super().__str__()

In [6]:
# And a game!
class Game:
    def __init__(self,*players):
        self.players = list(players)
        self.deck = Deck()
        self.deck.shuffle()
        self.deal()
        self.round = 1
        
    def deal(self):
        for _ in range(2):
            for player in self.players:
                player.take_card(self.deck.draw())
                
    def round_p(self):
        print("="*50)
        print(f"Round {self.round}")
        print("-"*50)
        for player in game.players:
            print(player)
        hit_list = []
        for player in self.players:
            if isinstance(player,CPU):
                player.count_cards(self)
            hit = player.hit()
            if hit:
                player.take_card(self.deck.draw())
                if sum(player.hand) > 21:
                    return False
            hit_list.append(hit)
        self.round +=1
        return any(hit_list)
    
    def bust(self,hand):
        return sum(hand) > 21
    
    def reveal(self):
        print("~"*50)
        print("REVEAL".ljust(25," "))
        print("~"*50)
        for player in game.players:
            if isinstance(player,CPU):
                print(player.reveal())
            else:
                print(player)
        
    
    def result(self):
        bust = [self.bust(player.hand) for player in self.players]
        print("~"*50)
        print("RESULT".ljust(25," "))
        print("~"*50)
        if any(bust):
            print(f"{self.players[bust.index(True)].name} busted!")
            print(f"{self.players[bust.index(False)].name} won!")
        else:
            hands = [sum(player.hand) for player in self.players]
            win = hands.index(max(hands))
            print(f"{self.players[win].name} won!")
            
    
    def play(self):
        cont = True
        while cont:
            cont = self.round_p()
        self.reveal()
        self.result()

In [7]:
p = Player()
cpu = CPU()
game = Game(p,cpu)

In [8]:
game.play()

Round 1
--------------------------------------------------
Player
┌───┐┌───┐
| 4♣|| 7♣|
└───┘└───┘
CPU
┌───┐┌───┐
|▓▓▓|| A♣|
└───┘└───┘
Player, do you want another card? [y|n]y
CPU, do you want another card? [y|n]
Yes
Round 2
--------------------------------------------------
Player
┌───┐┌───┐┌───┐
| 4♣|| 7♣|| 6♥|
└───┘└───┘└───┘
CPU
┌───┐┌───┐┌───┐
|▓▓▓|| A♣|| K♥|
└───┘└───┘└───┘
Player, do you want another card? [y|n]y
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
REVEAL                   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Player
┌───┐┌───┐┌───┐┌───┐
| 4♣|| 7♣|| 6♥|| 7♥|
└───┘└───┘└───┘└───┘
CPU
┌───┐┌───┐┌───┐
| 5♦|| A♣|| K♥|
└───┘└───┘└───┘
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
RESULT                   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Player busted!
CPU won!
